In [ ]:
%cd ../data

In [ ]:
from huggingface_hub import HfApi

In [ ]:
DATASET_NAMES = ["property_prediction","molgendata", "synthesis_tasks",]

In [ ]:
import os
import time

api = HfApi()
for name in DATASET_NAMES[:]:
    os.system("rm {}.tar.gz".format(name))
    print("Deleted '{}', compressing again".format(name))
    t0 = time.time()
    os.system("tar -czf {}.tar.gz {}".format(name, name))
    t1 = time.time()
    print("Compressed '{}' in {:.2f}s".format(name, t1-t0))
    api.upload_file(
        path_or_fileobj="{}.tar.gz".format(name),
        path_in_repo="{}.tar.gz".format(name),
        repo_id="Franso/MolGenData",
        repo_type="dataset",
    )
    t2 = time.time()
    print("Uploaded '{}' in {:.2f}s".format(name, t2-t1))



In [ ]:
FILES = [
    "molgendata/test_data/test_prompts_ood_boxed.jsonl",
    "property_prediction/eval_prompts_boxed.jsonl",
    "synthesis_tasks/eval_prompts/eval_prompts_enamine.jsonl",
    "synthesis_tasks/eval_prompts/eval_prompts_chembl.jsonl"
]

In [ ]:
from datasets import Dataset
import pandas as pd

def iterable_from_dataset(dataset):
    for item in dataset:
        conv = item.conversations[0]
        yield {
            "id": conv.identifier,
            "message": {"role": conv.messages[0].role, "content": conv.messages[0].content},
        }

def to_hf_format(dataset):
    dict_dataset = {}
    for line in iterable_from_dataset(dataset):
        for k, v in line.items():
            if k not in dict_dataset:
                dict_dataset[k] = []
            dict_dataset[k].append(v)

    return Dataset.from_dict(dict_dataset)

In [ ]:
DATASET_NAMES = ["property_prediction","molgendata", "synthesis_tasks",]

for name in DATASET_NAMES[:]:
    path = Path(name)
    # Upload training datasets
    for data_path in path.rglob("train_prompts*.jsonl"):
        print(name, data_path)
        dataset = read_jsonl(data_path)
        hf_dataset = to_hf_format(dataset)
        hf_dataset.push_to_hub(
            data_dir="{}_hf".format(name),
            repo_id="Franso/MolGenData",
            split="train"
        )

In [ ]:
DATASET_NAMES = ["property_prediction","molgendata", "synthesis_tasks",]
for name in DATASET_NAMES[:]:
    path = Path(name)
    # Upload training datasets
    for data_path in path.rglob("eval_prompts*.jsonl"):
        print(name, data_path)
        dataset = read_jsonl(data_path)
        hf_dataset = to_hf_format(dataset)
        hf_dataset.push_to_hub(
            data_dir="{}_hf".format(name),
            repo_id="Franso/MolGenData",
            split="validation"
        )